Ce notebook a été créé pour les requètes via les API.

# Importation des modules

In [3]:
import pandas as pd
import numpy as np
import requests
import searchtweets
from datetime import *

# Récupération des tweets via l'API de Twitter

In [7]:
premium_search_args = searchtweets.load_credentials("twitter_keys_idrissa.yaml",
                                       yaml_key="search_tweets_api",
                                       env_overwrite=False)

# Format de date_debut et date_fin : '2021-3-1'

def tweets_entre_2dates(date_debut, date_fin):
    rule = searchtweets.gen_rule_payload("(pfizer OR moderna OR astrazeneca OR janssen) lang:fr place_country:FR", 
                                         results_per_call=100, from_date=date_debut, to_date=date_fin) 

    rs = searchtweets.ResultStream(rule_payload=rule,
                      max_results=100,
                      **premium_search_args)

    return list(rs.stream())

def csv_tweets_entre2dates(date_debut, date_fin):
    tweets = tweets_entre_2dates(date_debut, date_fin)
    clean_tweets = []
    fields = ['id', "date", 'screenName', "text", "type"]
    for tweet in tweets:
        clean_tweet = [tweet.id, datetime.fromtimestamp(tweet.created_at_seconds), tweet.screen_name, tweet.all_text, tweet.tweet_type]
        clean_tweets.append(clean_tweet)

    tweets_df = pd.DataFrame(clean_tweets)
    tweets_df.columns = fields
    # globals()[f"my_variable_{i}"] = i
    tweets_df.to_csv('projetpy/fichiers_csv/tweets_'+date_debut+'_to_'+date_fin+'.csv', index = False, header=True)

In [9]:
csv_tweets_entre2dates('2021-3-1', '2021-4-1')

FileNotFoundError: [Errno 2] No such file or directory: 'projetpy/fichiers_csv/tweets_2021-3-1_to_2021-4-1.csv'

In [ ]:
dates = ["2021-"+str(i)+"-1" for i in range(3, 13)]

# Récupération des données de vaccination via l'API de AMELI 